In [39]:
import pandas as pd
import numpy as np

excel_file = 'Waterfall Actual vs Projected.xlsx'


xls = pd.ExcelFile(excel_file)
sheet_names = xls.sheet_names

# Reading the last sheet into a DataFrame
df = pd.read_excel(excel_file, sheet_name=sheet_names[-1])

In [40]:

# Creating 'Max_Magnitude' column
df['Max_Magnitude'] = df[['Actual', 'Projection']].abs().max(axis=1)

# Initializing Height column
df['Height'] = 0

# Calculating cumulative height
height = 0
for index, row in df.iterrows():
  if row['Actual'] > 0 or row['Projection'] > 0:
    height += row['Max_Magnitude']
  else:
    height -= row['Max_Magnitude']
  df.at[index, 'Height'] = height
df

,Particular,Actual,Projection,Max_Magnitude,Height
0,Opening Limit,700,700,700,700
1,Retirement LC,672,700,700,1400
2,Retirement BG,20,50,50,1450
3,Projection LC,-957,-900,957,493
4,Projection BG,-645,-700,700,-207
5,Closing Limit,-910,-850,910,-1117


In [41]:
# Calculating Stack0 values
df['Stack0'] = 0
cumsum = 0
for index, row in df.iterrows():
  if row['Actual'] > 0 or row['Projection'] > 0:
    df.at[index, 'Stack0'] = cumsum
    cumsum += row['Max_Magnitude']
  else:
    if row['Height'] > 0:
      df.at[index, 'Stack0'] = row['Height']
    
df

,Particular,Actual,Projection,Max_Magnitude,Height,Stack0
0,Opening Limit,700,700,700,700,0
1,Retirement LC,672,700,700,1400,700
2,Retirement BG,20,50,50,1450,1400
3,Projection LC,-957,-900,957,493,493
4,Projection BG,-645,-700,700,-207,0
5,Closing Limit,-910,-850,910,-1117,0


In [43]:
# Calculating Stack1 values
df['Stack1_A'] = 0
df['Stack1_P'] = 0
for index, row in df.iterrows():
  if row['Actual'] > 0 or row['Projection'] > 0:
    df.at[index, 'Stack1_A'] = row['Actual']
    df.at[index, 'Stack1_P'] = row['Projection']
  else:
    if row['Height'] > 0:
      df.at[index, 'Stack1_A'] = abs(row['Actual'])
      df.at[index, 'Stack1_P'] = abs(row['Projection'])
    elif row['Height'] < 0 and df.at[index-1, 'Stack0'] > 0:
      df.at[index, 'Stack1_A'] = df.at[index-1, 'Stack0']
      df.at[index, 'Stack1_P'] = df.at[index-1, 'Stack0']
df

,Particular,Actual,Projection,Max_Magnitude,Height,Stack0,Stack1_A,Stack1_P
0,Opening Limit,700,700,700,700,0,700,700
1,Retirement LC,672,700,700,1400,700,672,700
2,Retirement BG,20,50,50,1450,1400,20,50
3,Projection LC,-957,-900,957,493,493,957,900
4,Projection BG,-645,-700,700,-207,0,493,493
5,Closing Limit,-910,-850,910,-1117,0,0,0


In [45]:
# Calculating Stack-2 values
df['Stack-2_A'] = 0
df['Stack-2_P'] = 0
for index, row in df.iterrows():
  if row['Height'] < 0 and df.at[index-1, 'Stack0'] > 0:
    df.at[index, 'Stack-2_A'] = -(abs(row['Actual']) - df.at[index, 'Stack1_A'])
    df.at[index, 'Stack-2_P'] = -(abs(row['Projection']) - df.at[index, 'Stack1_P'])
  elif row['Height'] < 0 and df.at[index-1, 'Stack0'] == 0:
    df.at[index, 'Stack-2_A'] = row['Actual']
    df.at[index, 'Stack-2_P'] = row['Projection']
df

,Particular,Actual,Projection,Max_Magnitude,Height,Stack0,Stack1_A,Stack1_P,Stack-2_A,Stack-2_P
0,Opening Limit,700,700,700,700,0,700,700,0,0
1,Retirement LC,672,700,700,1400,700,672,700,0,0
2,Retirement BG,20,50,50,1450,1400,20,50,0,0
3,Projection LC,-957,-900,957,493,493,957,900,0,0
4,Projection BG,-645,-700,700,-207,0,493,493,-152,-207
5,Closing Limit,-910,-850,910,-1117,0,0,0,-910,-850


In [46]:
# Calculating Stack-1 values
df['Stack-1_A'] = 0
df['Stack-1_P'] = 0
cum_stack_minus1_A = 0
for index, row in df.iterrows():
  if row['Height'] < 0 and df.at[index-1, 'Stack0'] == 0:
    cum_stack_minus1_A += -(max(abs(df.at[index-1, 'Stack-2_A']), abs(df.at[index-1, 'Stack-2_P'])))
    df.at[index, 'Stack-1_A'] = cum_stack_minus1_A
    df.at[index, 'Stack-1_P'] = df.at[index, 'Stack-1_A']

df

,Particular,Actual,Projection,Max_Magnitude,Height,Stack0,Stack1_A,Stack1_P,Stack-2_A,Stack-2_P,Stack-1_A,Stack-1_P
0,Opening Limit,700,700,700,700,0,700,700,0,0,0,0
1,Retirement LC,672,700,700,1400,700,672,700,0,0,0,0
2,Retirement BG,20,50,50,1450,1400,20,50,0,0,0,0
3,Projection LC,-957,-900,957,493,493,957,900,0,0,0,0
4,Projection BG,-645,-700,700,-207,0,493,493,-152,-207,0,0
5,Closing Limit,-910,-850,910,-1117,0,0,0,-910,-850,-207,-207


In [47]:
# Creating PIndex
df['PIndex'] = df.index
df = df.set_index('PIndex')

df

,Particular,Actual,Projection,Max_Magnitude,Height,Stack0,Stack1_A,Stack1_P,Stack-2_A,Stack-2_P,Stack-1_A,Stack-1_P
PIndex,,,,,,,,,,,,
0,Opening Limit,700,700,700,700,0,700,700,0,0,0,0
1,Retirement LC,672,700,700,1400,700,672,700,0,0,0,0
2,Retirement BG,20,50,50,1450,1400,20,50,0,0,0,0
3,Projection LC,-957,-900,957,493,493,957,900,0,0,0,0
4,Projection BG,-645,-700,700,-207,0,493,493,-152,-207,0,0
5,Closing Limit,-910,-850,910,-1117,0,0,0,-910,-850,-207,-207


In [48]:

# Removing unnecessary columns
df = df.drop(['Actual', 'Projection', 'Max_Magnitude', 'Height'], axis=1)

# Duplicating Stack0 and rename
df['Stack0_P'] = df['Stack0']
df = df.rename(columns={'Stack0': 'Stack0_A'})

df


,Particular,Stack0_A,Stack1_A,Stack1_P,Stack-2_A,Stack-2_P,Stack-1_A,Stack-1_P,Stack0_P
PIndex,,,,,,,,,
0,Opening Limit,0,700,700,0,0,0,0,0
1,Retirement LC,700,672,700,0,0,0,0,700
2,Retirement BG,1400,20,50,0,0,0,0,1400
3,Projection LC,493,957,900,0,0,0,0,493
4,Projection BG,0,493,493,-152,-207,0,0,0
5,Closing Limit,0,0,0,-910,-850,-207,-207,0


In [49]:

# Extracting A and P values into separate DataFrames
df_a = df[['Particular', 'Stack0_A', 'Stack1_A', 'Stack-1_A', 'Stack-2_A']].rename(columns={'Stack0_A': 'Stack0', 'Stack1_A': 'Stack1', 'Stack-1_A': 'Stack-1', 'Stack-2_A': 'Stack-2'})
df_p = df[['Particular', 'Stack0_P', 'Stack1_P', 'Stack-1_P', 'Stack-2_P']].rename(columns={'Stack0_P': 'Stack0', 'Stack1_P': 'Stack1', 'Stack-1_P': 'Stack-1', 'Stack-2_P': 'Stack-2'})

# Adding a column to indicate A or P
df_a['Type'] = 'A'
df_p['Type'] = 'P'

# Concatenating the DataFrames
df_combined = pd.concat([df_a, df_p])


df_combined


,Particular,Stack0,Stack1,Stack-1,Stack-2,Type
PIndex,,,,,,
0,Opening Limit,0,700,0,0,A
1,Retirement LC,700,672,0,0,A
2,Retirement BG,1400,20,0,0,A
3,Projection LC,493,957,0,0,A
4,Projection BG,0,493,0,-152,A
5,Closing Limit,0,0,-207,-910,A
0,Opening Limit,0,700,0,0,P
1,Retirement LC,700,700,0,0,P
2,Retirement BG,1400,50,0,0,P


In [7]:
df_combined.to_csv('Matrix.csv')